# Conexion api (API del mercado de valores)

In [214]:
import requests
import pandas as pd

# Leer la clave de API desde el archivo
with open("C:/Users/betoo/Downloads/api_key.txt", "r") as file:
    access_key = file.read().strip()  # Lee la clave y elimina cualquier espacio en blanco

# Definir la URL de la API
url = f"https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol=TSCO.LON&apikey={access_key}"

# Realizar la solicitud a la API
response = requests.get(url)

if response.status_code == 200:
    data = response.json()

    if "Monthly Adjusted Time Series" in data:
        # Crear un DataFrame de pandas a partir de los datos
        df = pd.DataFrame(data["Monthly Adjusted Time Series"]).T

        # Renombrar las columnas
        df = df.rename(columns={
            "1. open": "OPEN1",
            "2. high": "HIGH",
            "3. low": "LOW",
            "4. close": "CLOSE1",
            "5. adjusted close": "A.CLOSE",
            "6. volume": "VOLUME",
            "7. dividend amount": "D.AMOUNT"
        })

        print(df)
     # Guardar los datos en un archivo CSV
        df.to_csv("datos_excel.csv", index=False)

        print("Los datos se han guardado en 'datos_excel.csv'.")
    else:
        print("No se encontraron datos de la serie mensual ajustada.")
else:
    print("La solicitud a la API no fue exitosa. Código de estado:", response.status_code)


               OPEN1      HIGH       LOW    CLOSE1   A.CLOSE     VOLUME  \
2023-11-06  272.3000  279.8000  269.0000  275.9000  275.9000   65867001   
2023-10-31  266.1000  283.6000  258.6000  269.7000  269.7000  536534298   
2023-09-29  266.6000  274.8000  254.1000  264.2000  260.5815  429461938   
2023-08-31  257.9000  267.7000  244.5000  265.7000  262.0609  406863130   
2023-07-31  250.9000  264.8000  244.2000  258.0000  254.4664  432634258   
...              ...       ...       ...       ...       ...        ...   
2005-06-30  312.7501  323.7501  310.5001  318.7501  203.0794  493142683   
2005-05-31  310.0001  319.2501  303.0001  313.0001  199.4160  430252527   
2005-04-29  316.0001  328.0001  304.2501  308.0001  196.2305  522330401   
2005-03-31  306.5001  322.5001  303.0001  316.5001  198.3586  583146220   
2005-02-28  308.5001  316.5001  301.5001  305.2501  191.3080  547917183   

           D.AMOUNT  
2023-11-06   0.0000  
2023-10-31   3.8500  
2023-09-29   0.0000  
2023-08-31 

In [215]:
df.shape

(226, 7)

In [216]:
df.dtypes

OPEN1       object
HIGH        object
LOW         object
CLOSE1      object
A.CLOSE     object
VOLUME      object
D.AMOUNT    object
dtype: object

In [217]:
df.isnull().sum()

OPEN1       0
HIGH        0
LOW         0
CLOSE1      0
A.CLOSE     0
VOLUME      0
D.AMOUNT    0
dtype: int64

# Conexion a Redshift

In [225]:
import psycopg2

url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws"
data_base="data-engineer-database"
user="betooosao78_coderhouse"
with open("C:/Users/betoo/Downloads/contraseña.txt","r") as f:
    pwd = f.read()

In [226]:

try:
    conn = psycopg2.connect(
       host ='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
       dbname=data_base,
       user=user,
       password=pwd,
       port='5439'
    )
    print("Connected to Redshift successfully!")
    
except Exception as e:
    print("Unable to conection to Redshift")
    print(e)

Connected to Redshift successfully!


In [227]:
 conn

<connection object at 0x00000170B3138E10; dsn: 'user=betooosao78_coderhouse password=xxx dbname=data-engineer-database host=data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com port=5439', closed: 0>

# Verificar que la tabla exista

In [228]:
cur = conn.cursor()
cur.execute("SELECT * FROM monthly_time")
resultado = cur.fetchall()
resultado

[('272.3000',
  '279.8000',
  '269.0000',
  '275.9000',
  '275.9000',
  '65867001',
  '0.0000'),
 ('266.1000',
  '283.6000',
  '258.6000',
  '269.7000',
  '269.7000',
  '536534298',
  '3.8500'),
 ('266.6000',
  '274.8000',
  '254.1000',
  '264.2000',
  '260.5815',
  '429461938',
  '0.0000'),
 ('257.9000',
  '267.7000',
  '244.5000',
  '265.7000',
  '262.0609',
  '406863130',
  '0.0000'),
 ('250.9000',
  '264.8000',
  '244.2000',
  '258.0000',
  '254.4664',
  '432634258',
  '0.0000'),
 ('261.6000',
  '267.3000',
  '245.6000',
  '248.4000',
  '244.9979',
  '442697816',
  '0.0000'),
 ('280.4000',
  '285.3000',
  '260.0000',
  '260.8000',
  '257.2280',
  '424360362',
  '7.0500'),
 ('265.7000',
  '282.2000',
  '262.0400',
  '281.1000',
  '270.2165',
  '304714026',
  '0.0000'),
 ('253.0000',
  '307.2251',
  '244.1000',
  '265.7000',
  '255.4127',
  '496212270',
  '0.0000'),
 ('246.3000',
  '257.1000',
  '240.4000',
  '255.0000',
  '245.1270',
  '435519782',
  '0.0000'),
 ('225.4000',
  '251.

In [223]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# Configura los parámetros de conexión a Amazon Redshift
host = 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
database = 'data-engineer-database'
user = 'betooosao78_coderhouse'
password = '3sNMN32zqI'
port = '5439'  # El puerto predeterminado de Redshift

def enviar_a_redshift(df):
    try:
        # Conéctate a Redshift
        conn = psycopg2.connect(
            host=host,
            database=database,
            user=user,
            password=password,
            port=port
        )

        cursor = conn.cursor()

        # Define el nombre de la tabla en Redshift
        table_name = 'monthly_time'

        # Crea una conexión de SQLAlchemy a Redshift
        engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

        # Carga los datos del DataFrame en la tabla de Redshift
        df.to_sql(table_name, engine, if_exists='replace', index=False)

        conn.commit()
        print(f"Los datos se han enviado a la tabla {table_name} en Redshift.")
    except Exception as e:
        print(f"Ocurrió un error: {e}")
    finally:
        if conn:
            conn.close()

# Luego de cargar los datos en el DataFrame
# Puedes llamar la función enviar_a_redshift para enviar los datos a Redshift
enviar_a_redshift(df)



Los datos se han enviado a la tabla monthly_time en Redshift.
